# UPMCE Machine Learning Palooza
### Goal
Predict the length-of-stay (LOS) of a patient in a hospital. See this repo to get details about the challenge [here](https://git.tdc.upmc.edu/MachineLearning/ml-intro-spark-regression) 

#### 1) Get my features before splitting

In [1]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from ohe import one_hot_dataframe

In [2]:
url = 'http://sparkdl04:50070/webhdfs/v1/palooza/data/visit_train_panda.csv?op=OPEN'
source_data = pd.read_csv(url)
source_data.head()

,VisitID,Hospital,Dept_Code,PaymentType,Age,Race,Gender,FC,ArriveDate,DischargeDate,LOS,DXCODE,Description,DispenseID,DOC
0,HdBgCT1YkEl14280,SHY,436,I,77,W,M,MS,10/07/2014,10/10/2014,3,197.7,SECOND MALIG NEO LIVE,1,10028
1,HdBgCT1YkEl14280,SHY,436,I,77,W,M,MS,10/07/2014,10/10/2014,3,276.2,ACIDOSIS,1,10028
2,HdBgCT1YkEl14280,SHY,436,I,77,W,M,MS,10/07/2014,10/10/2014,3,198.5,SECONDARY MALIG NEO B,1,10028
3,HdBgCT1YkEl14280,SHY,436,I,77,W,M,MS,10/07/2014,10/10/2014,3,253.6,NEUROHYPOPHYSIS DIS N,1,10028
4,HdBgCT1YkEl14280,SHY,436,I,77,W,M,MS,10/07/2014,10/10/2014,3,V58.66,LONG-TERM USE ASPIRIN,1,10028


## Part A - Exclude DXCODE
DXCODE has over 4K unique values so it is impossible to encode using a dense matric like panda on a single machine.

In [3]:
from ohe import one_hot_dataframe
feature_names = ['LOS', 'Age', 'Race', 'Gender', 'Hospital', 'ArriveDate', 'DischargeDate']
source_data_features = source_data[feature_names]

source_data_features.ArriveDate = source_data_features.ArriveDate.map(lambda d: pd.to_datetime(str(d)))
source_data_features.DischargeDate = source_data_features.DischargeDate.map(lambda d: pd.to_datetime(str(d)))
source_data_features.DischargeDate.head()

c:\program files\anaconda2.3.0\lib\site-packages\pandas\core\generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


0   2014-10-10
1   2014-10-10
2   2014-10-10
3   2014-10-10
4   2014-10-10
Name: DischargeDate, dtype: datetime64[ns]

In [4]:
#Figure out how to get the month.
categorical_features = ['Race', 'Gender', 'ArriveMonth', 'DischargeMonth']

#### Let's see what happnes if I only encode DXCODE

In [5]:
_,df_cat,_ = one_hot_dataframe(source_data_features, categorical_features, True)
df_cat.head()

KeyError: "['ArriveMonth' 'DischargeMonth'] not in index"

In [10]:
print ('%d of the %d observations have unique DXCODES.' % 
       (len(source_data_features.DXCODE.unique()), len(source_data_features.index)))

4789 of the 279808 observations have unique DXCODES.
